In [ ]:
import pandas as pd

from estival.wrappers.nevergrad import optimize_model

from tb_incubator.plotting import plot_model_vs_actual
from tb_incubator.input import load_targets, load_param_info
from tb_incubator.calibrate import get_bcm

from multiprocessing import cpu_count
pd.options.plotting.backend = "plotly"

In [2]:
params = load_param_info()["value"]
all_targets = load_targets()

In [3]:
bcm = get_bcm(params)

In [4]:
def calibrate_model_with_optimisation(bcm):
    """
    This function performs a model calibration using optimisation. 
    Calibration is performed using the estival package, which implements a wrapper for optimisation methods provided by the nevergrad package. 

    Args:
        bcm: the calibration model object (type BayesianCompartmentalModel) 
    """

    from nevergrad.optimization.differentialevolution import TwoPointsDE
    orunner = optimize_model(bcm, opt_class=TwoPointsDE, num_workers=cpu_count(), budget=4000)
    rec = orunner.minimize(4000)
    optimised_params = rec.value[1]    
   
    return optimised_params

In [ ]:
optimised_params = calibrate_model_with_optimisation(bcm)
optimised_params

In [6]:
# run the modle with the optimised parameter set
res = bcm.run(optimised_params)

In [7]:
outs = res.derived_outputs

In [ ]:
plot = plot_model_vs_actual(
    outs, all_targets['notif'], "notification", "Notification", "Modelled vs Data", "Actual data"
)

plot.update_xaxes(range=[1978, 2023])
plot

In [ ]:
plot = plot_model_vs_actual(
    outs, all_targets['prevalence'], "prevalence", "Prevalence", "Modelled vs Data", "Actual data"
)

#plot.update_xaxes(range=[1978, 2023])
plot

In [ ]:
fig = outs["percentage_latent"].plot()
#set_plot_label(fig, indicator_names, "value")
#display_plot(fig, "percentage_latent", "svg")

fig